In [1]:
# !pip3 install pyenchant

In [2]:
import enchant
d = enchant.Dict("en_US")

In [3]:
d.check("Hello")

True

In [4]:
# !wget https://raw.githubusercontent.com/huseinzol05/malay-dataset/master/normalization/en-lexicon/en-social-media-lexicon.json

In [5]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [6]:
with open('/home/husein/translation/zsm_Latn.dev') as fopen:
    left = fopen.read().split('\n')
    
with open('/home/husein/translation/eng_Latn.dev') as fopen:
    right = fopen.read().split('\n')

In [7]:
len(left), len(right)

(998, 998)

In [8]:
import malaya

/home/husein/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [9]:
from malaya.text.rules import rules_normalizer, rules_compound_normalizer
from malaya.text.normalization import _is_number_regex
from collections import defaultdict
import re
import random
import json

In [10]:
with open('en-social-media-lexicon.json') as fopen:
    en_lexicon = json.load(fopen)

In [11]:
rev_rules_compound_normalizer = defaultdict(list)
for k, v in rules_compound_normalizer.items():
    rev_rules_compound_normalizer[v].append(k)

In [12]:
rev_rules_normalizer = defaultdict(list)
for k, v in rules_normalizer.items():
    rev_rules_normalizer[v].append(k)

In [13]:
rules_compound_normalizer_regex = (
    '(?:' + '|'.join(list(rev_rules_compound_normalizer.keys())) + ')'
)

In [14]:
# def _replace_compound(string, 
#                       rules_regex = rules_compound_normalizer_regex, 
#                       rules = rev_rules_compound_normalizer):
#     results = re.findall(rules_regex, string, flags=re.IGNORECASE
#     )
#     for r in results:
#         try:
#             string = string.replace(r, random.choice(rules[r.lower()]))
#         except:
#             pass
#     return string

def _replace_compound(string, rules = rev_rules_compound_normalizer):
    for k in list(rules.keys()):
        results = [(m.start(0), m.end(0)) for m in re.finditer(k, string, flags=re.IGNORECASE)]
        for r in results:
            sub = string[r[0]: r[1]]
            try:
                replaced = random.choice(rules[sub.lower()])
                if replaced:
                    if r[1] < len(string) and string[r[1]] != ' ':
                        continue
                    if r[0] - 1 > len(string) and string[r[0] - 1] != ' ':
                        continue

                    sub = case_of(sub)(replaced)
                    string = string[:r[0]] + sub + string[r[1]:]
            except:
                pass
    return string

In [15]:
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

def case_of(text):
    return (
        str.upper
        if text.isupper()
        else str.lower
        if text.islower()
        else str.title
        if text.istitle()
        else str
    )

def strip_punct(word):
    left = []
    right = []
    i = 0
    while i < len(word):
        if word[i] in PUNCTUATION:
            left.append(word[i])
            i += 1
        else:
            break
    i = len(word) - 1
    while i > 0:
        if word[i] in PUNCTUATION:
            right.append(word[i])
            i -= 1
        else:
            break
    left = ''.join(left)
    right = ''.join(right[::-1])
    if len(right):
        word_ = word[:-len(right)]
    else:
        word_ = word
    word_ = word_[len(left):]
    return left, right, word_


def replace_shortword(word, rules = rev_rules_normalizer):
    left, right, word_ = strip_punct(word)
    word_ = word_[len(left):]
    lower_word_ = word_.lower()
    if lower_word_ in rules:
        word_ = case_of(word_)(random.choice(rules[lower_word_]))
        word_ = f'{left}{word_}{right}'
        return word_
    else:
        return word
    
strip_punct('counters,')

('', ',', 'counters')

In [16]:
string = 'saya sangat cantik"'
[replace_shortword(word) for word in string.split()]

['uwee', 'bettt', 'cntik"']

In [17]:
[replace_shortword(word) for word in left[1].split()]

['Penyelidik',
 'utama',
 'mengatakan',
 'bahawa',
 'ia',
 'mgkn',
 'menghasilkan',
 'pengesanan',
 'awl',
 'kanser,',
 'tuberkulosis,',
 'HIV',
 'dn',
 'malaria',
 'kpd',
 'pesakit',
 'd',
 'negara-negara',
 'berpendapatan',
 'low,',
 'kat',
 'mn',
 'kadar',
 'kemandirian',
 'tuk',
 'penyakit',
 'sprt',
 'kanser',
 'payu',
 'dara',
 'ialah',
 'separuh',
 'dripda',
 'dhi',
 'negara-negara',
 'yg',
 'lbih',
 'kyy.']

In [18]:
[replace_shortword(word, rules = en_lexicon) for word in 'do you like chicken'.split()]

['do', 'yew', 'lik', 'chicken']

In [19]:
def replace_words_punct(left_word, right_word):
    left_left, left_right, left_word = strip_punct(left_word)
    right_left, right_right, right_word = strip_punct(right_word)
    return f'{left_left}{right_word}{left_right}'

def random_replace_alignment(left, right, alignment, min_replace = 2, max_replace = 7):
    splitted_left = left.split()
    splitted_right = right.split()
    
    selected_alignment = []
    for s in alignment:
        l = s[0]
        r = s[1]
        if _is_number_regex(splitted_left[l].replace(',', '').replace('.', '')) or _is_number_regex(splitted_right[r].replace(',', '').replace('.', '')):
            continue
        elif splitted_left[l].isupper() or splitted_right[r].isupper():
            continue
        elif splitted_left[l] == splitted_right[r]:
            continue
        elif splitted_right[r].lower() in ['the', 'a', 'an', 'it', 'is', 'are']:
            continue
        else:
            selected_alignment.append((l, r))
    
    count_replace = random.randint(min_replace, min(max_replace, len(selected_alignment)))
    
    selected = random.sample(selected_alignment, count_replace)
    for s in selected:
        splitted_left[s[0]] = replace_words_punct(splitted_left[s[0]], splitted_right[s[1]])
    
    return ' '.join(splitted_left)

In [20]:
# random replace alignment
# random replace compound
# random replace word

In [21]:
random.random()

0.14077332551265032

In [22]:
malaya.augmentation.socialmedia_form('Saya')

['Sayak', 'Saye']

In [23]:
malaya.augmentation.vowel_alternate('saya')

'saya'

In [24]:
d.check("Hello")

True

In [25]:
import copy

consonants = 'bcdfghjklmnpqrstvwxyz'

def augment(left, p_replace_shortword = 0.7, p_socialmedia = 0.85, p_vowel_alternate = 0.5, p_replace_en_shortword = 0.3):
    
    left = _replace_compound(left, rules = copy.deepcopy(rev_rules_compound_normalizer))
    left = [replace_shortword(word, rules = en_lexicon) if random.random() > p_replace_en_shortword else word for word in left.split()]
    left = [(replace_shortword(word), False) if random.random() > p_replace_shortword else (word, True) for word in left]
    left_ = []
    for l in left:
        if _is_number_regex(l[0].replace(',', '').replace('.', '')):
            left_.append(l[0])
            continue
        if l[0].isupper():
            left_.append(l[0])
            continue
        if l[0].istitle():
            left_.append(l[0])
            continue
        if d.check(l[0]):
            left_.append(l[0])
            continue
        
        if l[1]:
            if random.random() > p_socialmedia:
                r = malaya.augmentation.socialmedia_form(l[0])
                if len(r):
                    s = random.choice(r)
                else:
                    s = l[0]
            else:
                s = l[0]
            if random.random() > p_vowel_alternate:
                s = malaya.augmentation.vowel_alternate(s)
                if random.random() and s[-1] == 'a' and s[-2] in consonants:
                    s = s[:-1] + 'e'
        else:
            s = l[0]
        left_.append(case_of(l[0])(s))
    return ' '.join(left_)

In [26]:
_replace_compound('ke', rules = copy.deepcopy(rev_rules_compound_normalizer))

'ke eh'

In [27]:
augment(left[0])

'Pd hari Isnin, Saintis daripadak Sekolah Perubatan Universiti Stamford mengumumkan penemuan alat diagnostik bru yang bsa mengasingkan sel-sel mengikut jenis: cip kcil yang bole dictak yang boleh dihasilakn menggunakan pencetak standard inkjet for kira-kira stu sen A.S setiap satu.'

In [28]:
fast_text = malaya.language_detection.fasttext()

In [29]:
from tqdm import tqdm

new_left, new_right = [], []
for i in tqdm(range(len(left))):
    if len(left[i].split()) > 100 or len(right[i].split()) > 100:
        continue
    langs = fast_text.predict([left[i], right[i]])
    if langs[0] == 'eng':
        continue
    if langs[1] == 'malay':
        continue
    try:
        new_left_ = augment(left[i])
        if new_left_ != left[i]:
            new_left.append(new_left_)
            new_right.append(right[i])
    except Exception as e:
        pass

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 998/998 [00:00<00:00, 3565.44it/s]


In [30]:
new_left

['Pada day Mon, Saintis darpade Skolah Perubatan Universiti Stamford mngmumkn penemuan alat diagnostk baru yanckh boleh mngasngkn sel-sel mengikut jnis: cip kecil yang blh dicetak yang boleh dihasilakn menggunakan pencetak standard inkjet untuk kra-kre atu sen A.S setiap satu.',
 'Penyelidik utama mengatakan bahawa ia mungkn menghasilkan pengesanan awal knsr, tuberkulosis, HIV dn malaria kpd pesakit di negara-negara berpndaptan rndah, di mane kadar kemandirian untuk penyakit sperti kanser payu dare ialah separuh daripada di negara-ngre yang lebh kyy.',
 'JAS 39C Gripen telah terhempas ke eh atas lndasan skitar jam 9:30 pgi waktu tempatan (0230 UTC) dn mletp, mengakibatkan lpangan trbng dtutp bgi airlines komersial.',
 'Juruterbang telah dikenal pasti sbgai Leader Team Dilokrit Pattavee.',
 'Media tempatan ngelaporin kenderaan api lapngn terbang terguling ketika mmberi mklum balas.',
 'Vidal yang berusia 28 tahun tlah menyertai pasukan Barça tiga musim yng lalu, drii Sevilla.',
 'Sejak 

In [31]:
left

['Pada hari Isnin, Saintis daripada Sekolah Perubatan Universiti Stamford mengumumkan penemuan alat diagnostik baru yang boleh mengasingkan sel-sel mengikut jenis: cip kecil yang boleh dicetak yang boleh dihasilakn menggunakan pencetak standard inkjet untuk kira-kira satu sen A.S setiap satu.',
 'Penyelidik utama mengatakan bahawa ia mungkin menghasilkan pengesanan awal kanser, tuberkulosis, HIV dan malaria kepada pesakit di negara-negara berpendapatan rendah, di mana kadar kemandirian untuk penyakit seperti kanser payu dara ialah separuh daripada di negara-negara yang lebih kaya.',
 'JAS 39C Gripen telah terhempas ke atas landasan sekitar jam 9:30 pagi waktu tempatan (0230 UTC) dan meletup, mengakibatkan lapangan terbang ditutup bagi penerbangan komersial.',
 'Juruterbang telah dikenal pasti sebagai Ketua Pasukan Dilokrit Pattavee.',
 'Media tempatan melaporkan kenderaan api lapangan terbang terguling ketika memberi maklum balas.',
 'Vidal yang berusia 28 tahun telah menyertai pasukan

In [32]:
import json

with open('augmented-eng_Latn-zsm_Latn-noisy-ms-en-dev-set-v3.json', 'w') as fopen:
    json.dump({'en': new_right, 'ms': new_left}, fopen)